## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [4]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data


In [5]:
import pandas as pd

In [3]:
#connect to G-Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling/process-steps.csv", sep=',',
    usecols=[0,1,2,3,4,5,6],
    encoding="utf8")
df.head()
docs = df["step_title"].values.tolist()

In [7]:
docs= docs[0:1000]

# **Topic Modeling**



## Training

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>20_000 documents). It is advised to turn this off if you want to speed up the model. 


### Stop Words for Count Vectorizer

In [8]:
# get Greek stop_words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
greek_stopwords = stopwords.words('greek')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
new_words = ['της', 'τη', 'του', 'από']

for word in new_words:
  greek_stopwords.append(word)

### Initialize models & Training

In [10]:
from bertopic import BERTopic 
from sentence_transformers import SentenceTransformer # Embeddings
from umap import UMAP #Dimensionality reduction
from hdbscan import HDBSCAN #clustering
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

In [11]:
sentence_model = SentenceTransformer("all-mpnet-base-v2")
umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.1, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=greek_stopwords) #ngram_range combination of words
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)

In [12]:
topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model)
topics = topic_model.fit_transform(docs)

**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [13]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,292,-1_κεπ_πολίτη_ενημέρωση_στοιχείων,"[κεπ, πολίτη, ενημέρωση, στοιχείων, καταχώριση...",[Παράδοση πιστοποιητικού στον πολίτη από Δήμο ...
1,0,97,0_έλεγχος_έλεγχος αίτησης_χρήστη_ελέγχου,"[έλεγχος, έλεγχος αίτησης, χρήστη, ελέγχου, αί...","[Έλεγχος της αίτησης, Έλεγχος αίτησης, Έλεγχος]"
2,1,94,1_υπαλλήλου_γραφείο_κοινοβουλίου_απόσπασης,"[υπαλλήλου, γραφείο, κοινοβουλίου, απόσπασης, ...",[Αρνητική απάντηση στο αίτημα απόσπασης και δι...
3,2,78,2_πολιτών_μητρώου_μητρώο_μητρώο πολιτών,"[πολιτών, μητρώου, μητρώο, μητρώο πολιτών, μητ...",[Αναζήτηση πολίτη στο πληροφοριακό σύστημα του...
4,3,44,3_δικαιολογητικών_παραλαβή πιστοποιητικού_πιστ...,"[δικαιολογητικών, παραλαβή πιστοποιητικού, πισ...","[Παραλαβή πιστοποιητικού από τον Δήμο, Παραλαβ..."
5,4,36,4_πρωτοκόλληση_παραλαβή_παραλαβή αίτησης_αιτήμ...,"[πρωτοκόλληση, παραλαβή, παραλαβή αίτησης, αιτ...","[Παραλαβή αιτήματος και πρωτοκόλληση, Παραλαβή..."
6,5,35,5_πληρότητας_δικαιολογητικών_έλεγχος_έλεγχος π...,"[πληρότητας, δικαιολογητικών, έλεγχος, έλεγχος...","[Έλεγχος Πληρότητας Δικαιολογητικών, Έλεγχος π..."
7,6,30,6_ηλεκτρονική_ηλεκτρονική παραλαβή_ηλεκτρονική...,"[ηλεκτρονική, ηλεκτρονική παραλαβή, ηλεκτρονικ...",[Ηλεκτρονική Παραλαβή Αίτησης και Δικαιολογητι...
8,7,27,7_απόρριψη_απόρριψη αίτησης_απόρριψη αιτήματος...,"[απόρριψη, απόρριψη αίτησης, απόρριψη αιτήματο...","[Απόρριψη αιτήματος, Απόρριψη αιτήματος, Απόρρ..."
9,8,26,8_έλεγχος προϊστάμενο φυσική_προϊστάμενο φυσικ...,"[έλεγχος προϊστάμενο φυσική, προϊστάμενο φυσικ...",[Έλεγχος από τον Προϊστάμενο και φυσική υπογρα...


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [14]:
topic_model.get_topic(0)  # Select the most frequent topic

[('έλεγχος', 0.06625818463512957),
 ('έλεγχος αίτησης', 0.03833718620418648),
 ('χρήστη', 0.03232314832654789),
 ('ελέγχου', 0.03214999936158945),
 ('αίτησης έλεγχος', 0.02751001910436485),
 ('έγκριση', 0.02428524134361297),
 ('έλεγχος αίτησης έλεγχος', 0.02297046466220723),
 ('επιτυχής', 0.02297046466220723),
 ('αίτησης', 0.02270358282200368),
 ('επιτόπιος', 0.022588073348550515)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

In [15]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Φυσική ταυτοποίηση πολίτη στο ΚΕΠ,11,11_ταυτοποίηση_ταυτοποίηση πολίτη_φυσική ταυτο...,"[ταυτοποίηση, ταυτοποίηση πολίτη, φυσική ταυτο...","[Φυσική ταυτοποίηση πολίτη, Φυσική ταυτοποίηση...",ταυτοποίηση - ταυτοποίηση πολίτη - φυσική ταυτ...,1.000000,False
1,Υποβολή αίτησης στο Πληροφοριακό σύστημα,4,4_πρωτοκόλληση_παραλαβή_παραλαβή αίτησης_αιτήμ...,"[πρωτοκόλληση, παραλαβή, παραλαβή αίτησης, αιτ...","[Παραλαβή αιτήματος και πρωτοκόλληση, Παραλαβή...",πρωτοκόλληση - παραλαβή - παραλαβή αίτησης - α...,0.467905,False
2,Εκτύπωση αποδεικτικού φορολογικής ενημερότητας,16,16_πώλησης_κτηνιατρικών φαρμακευτικών_πώλησης ...,"[πώλησης, κτηνιατρικών φαρμακευτικών, πώλησης ...",[Παραλαβή άδειας χονδρικής πώλησης κτηνιατρικώ...,πώλησης - κτηνιατρικών φαρμακευτικών - πώλησης...,1.000000,False
3,Εκκίνηση διαδικασίας,12,12_διαδικασίας_εκκίνηση διαδικασίας_εκκίνηση_δ...,"[διαδικασίας, εκκίνηση διαδικασίας, εκκίνηση, ...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε...",διαδικασίας - εκκίνηση διαδικασίας - εκκίνηση ...,0.551730,True
4,Παραλαβή αιτήματος για διακίνηση χοιροειδών,-1,-1_κεπ_πολίτη_ενημέρωση_στοιχείων,"[κεπ, πολίτη, ενημέρωση, στοιχείων, καταχώριση...",[Παράδοση πιστοποιητικού στον πολίτη από Δήμο ...,κεπ - πολίτη - ενημέρωση - στοιχείων - καταχώρ...,0.000000,False
...,...,...,...,...,...,...,...,...
995,Εγγραφή του πλοίου στο e-Μητρώο Πλοίων από τον...,5,5_πληρότητας_δικαιολογητικών_έλεγχος_έλεγχος π...,"[πληρότητας, δικαιολογητικών, έλεγχος, έλεγχος...","[Έλεγχος Πληρότητας Δικαιολογητικών, Έλεγχος π...",πληρότητας - δικαιολογητικών - έλεγχος - έλεγχ...,0.430777,False
996,Παραλαβή αίτησης με φυσικό τρόπο και πρωτοκόλληση,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,1.000000,False
997,Χρέωση σε αρμόδιο Τμήμα,-1,-1_κεπ_πολίτη_ενημέρωση_στοιχείων,"[κεπ, πολίτη, ενημέρωση, στοιχείων, καταχώριση...",[Παράδοση πιστοποιητικού στον πολίτη από Δήμο ...,κεπ - πολίτη - ενημέρωση - στοιχείων - καταχώρ...,0.000000,False
998,Ανάθεση της αίτησης σε αρμόδιο υπάλληλο,4,4_πρωτοκόλληση_παραλαβή_παραλαβή αίτησης_αιτήμ...,"[πρωτοκόλληση, παραλαβή, παραλαβή αίτησης, αιτ...","[Παραλαβή αιτήματος και πρωτοκόλληση, Παραλαβή...",πρωτοκόλληση - παραλαβή - παραλαβή αίτησης - α...,0.557100,False


In [16]:
result = topic_model.get_document_info(docs)

In [ ]:
### Attributes

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

## Similarity

In [39]:
result.loc[result['Topic'] == 13]

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
71,Υποβολή Αίτησης με φυσικό τρόπο,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,1.000000,False
187,Υποβολή της αίτησης και δικαιολογητικών με φυσ...,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,0.827928,False
251,Υποβολή της Αίτησης με φυσικό τρόπο,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,1.000000,False
318,Υποβολή ψηφιακής Αίτησης μέσω Κ.Ε.Π.,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,0.960326,False
367,Παραλαβή της Αίτησης με φυσικό τρόπο,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,1.000000,True
379,Παραλαβή Αίτησης με φυσικό τρόπο,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,1.000000,True
390,Υποβολή Αίτησης με φυσικό τρόπο,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,1.000000,False
958,Παραλαβή της αίτησης και πρωτοκόλληση (με φυσι...,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,1.000000,False
959,Παραλαβή της αίτησης (με ψηφιακό τρόπο),13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,0.974475,False
968,Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτο...,13,13_τρόπο_φυσικό τρόπο_φυσικό_αίτησης φυσικό τρόπο,"[τρόπο, φυσικό τρόπο, φυσικό, αίτησης φυσικό τ...",[Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτ...,τρόπο - φυσικό τρόπο - φυσικό - αίτησης φυσικό...,0.895762,True


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

def get_sentence_embeddings(sentences):
    tokenizer = BertTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
    model = BertModel.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

    # Tokenize the sentences
    encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

    # Obtain the BERT embeddings
    with torch.no_grad():
        model_outputs = model(**encoded_inputs)

    # Extract the sentence embeddings from BERT's output
    sentence_embeddings = model_outputs.last_hidden_state[:, 0, :].numpy()

    return sentence_embeddings

def calculate_similarity(embedding1, embedding2):
    similarity = cosine_similarity([embedding1], [embedding2])
    return similarity[0][0]

def filter_similar_sentences(sentences, similarity_threshold=0.70):
    kept_sentences = [sentences[0]]
    kept_embeddings = [get_sentence_embeddings([sentences[0]])[0]]
    for i in range(1, len(sentences)):
        curr_embedding = get_sentence_embeddings([sentences[i]])[0]
        is_similar = False
        for j in range(len(kept_embeddings)):
            similarity = calculate_similarity(curr_embedding, kept_embeddings[j])
            if similarity >= similarity_threshold:
                is_similar = True
                break
        if not is_similar:
            kept_sentences.append(sentences[i])
            kept_embeddings.append(curr_embedding)
    return kept_sentences

In [40]:
# Example usage
sentences = list(result.loc[result['Topic'] == 13]["Document"])

kept_sentences = filter_similar_sentences(sentences, similarity_threshold=0.70)

Some weights of the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum

In [41]:
kept_sentences[0:10]

['Υποβολή Αίτησης με φυσικό τρόπο',
 'Υποβολή ψηφιακής Αίτησης μέσω Κ.Ε.Π.',
 'Παραλαβή της αίτησης και πρωτοκόλληση (με φυσικό τρόπο)',
 'Παραλαβή της αίτησης (με ψηφιακό τρόπο)',
 'Παραλαβή της αίτησης (με φυσικό τρόπο) - Πρωτοκόλληση']

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [42]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [43]:
topic_model.visualize_distribution(probs[200], min_probability=0.001)

NameError: ignored

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [44]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [45]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [46]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('του', 0.025664358552657897),
 ('υπάλληλο', 0.021681286210734525),
 ('αρμόδιο', 0.02159625460109819),
 ('τμήματος', 0.019105794065593856),
 ('τμήμα', 0.018704252370337602),
 ('προϊστάμενο', 0.018704252370337602),
 ('χρέωση', 0.018354433558546504),
 ('την', 0.016629128086533856),
 ('στην', 0.016082461581266524),
 ('προϊστάμενο του', 0.016073079262104578)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(docs, nr_topics=20)

2023-05-23 09:34:59,667 - BERTopic - Reduced number of topics from 49 to 20


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[2, 1, -1, 8, 6, 6, 0, 6, 6, 8, 0, 1, 0, 6, -1, 6, -1, 0, 5, 6, -1, 6, 8, 0, 0, 1, 0, 0, 0, -1, 0, 5, 5, 5, 1, 0, 17, 17, 17, 17, -1, -1, 17, 1, 11, 1, 0, 16, 16, 0, 3, -1, 0, 0, 0, 5, 0, 0, 8, 6, 6, 6, 6, 6, 2, 0, 18, 0, 0, 8, 1, 0, 0, 0, 1, 8, 0, 11, 4, 0, 0, -1, 0, 0, 4, -1, -1, 18, 1, 0, -1, 0, 0, -1, -1, 8, 0, 1, -1, 0, 5, 4, -1, 0, 5, -1, 0, 0, 5, 1, 0, 17, 17, 17, 17, 17, 0, -1, 17, 15, 1, 0, -1, 0, -1, 12, 0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 5, 3, 6, 6, -1, 3, 5, 3, 6, 6, 8, 0, 1, 0, 0, 12, 13, -1, 0, 0, 5, 8, 0, 1, -1, 1, 0, 4, -1, -1, -1, 0, 0, 0, 2, 0, 0, 0, 2, 4, 2, 0, 0, -1, 0, 0, 0, 0, 15, 8, 0, 1, 15, 15, 15, 1, -1, 0, -1, 6, 0, 8, -1, -1, 2, 0, -1, -1, -1, 4, -1, 0, -1, -1, 4, 4, 0, 0, 0, 5, 8, 0, 1, 0, 15, 0, 0, -1, 1, 13, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 5, 0, 0, 0, -1, -1, 0, 13, 1, 0, 0, 1, 5, 0, 0, 0, 11, 13, 8, 0, 1, 0, 6, -1, 6, -1, 0, 5, 6, -1, 6, 8, 0, 1, 0, 3, 3, 3, 6, -1, -1, -1, 4, 0, 5, -1, 4, 0, 0, 0, 0, 1, -1, -1, 0, -1, 0, 0, 1, -1, 8, 0, 1, 0,

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[14, 10, 11, 4, 0]

In [ ]:
topic_model.get_topic(0)

[('αίτησης', 0.034193058385162005),
 ('της', 0.028939271578152494),
 ('και', 0.027615165135388156),
 ('έλεγχος', 0.02500530017958265),
 ('του', 0.02423669796376737),
 ('βεβαίωσης', 0.021996993103581507),
 ('της αίτησης', 0.02164918737136251),
 ('από', 0.021125869087540137),
 ('παραλαβή', 0.0202417005623797),
 ('δικαιολογητικών', 0.019894030749705614)]